In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import h5py


In [9]:

# 데이터 불러오기
data = pd.read_csv('train.csv') 

# 입력과 출력 데이터 나누기
X = data.drop(['FloodProbability','id'], axis=1).values  # 입력 특성
y = data['FloodProbability'].values  # 출력 타겟

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# 모델 훈련
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, model_checkpoint])

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

# Test Loss: 0.0003612188156694174
# Test MAE: 0.014995004050433636
# 기본

# Test Loss: 0.0003593358560465276
# Test MAE: 0.014905383810400963
# best_model.h5 추가

# Test Loss: 0.00035975821083411574
# Test MAE: 0.01483593787997961
# adam -> rmsprop

# Test Loss: 0.00035775540163740516
# Test MAE: 0.0148658761754632
# batch size 32 -> 64

# Test Loss: 0.00035533090704120696
# Test MAE: 0.01483441423624754
# rmsprop -> adam

# Test Loss: 0.0003565888910088688
# Test MAE: 0.014893828891217709
# 뉴런 64 -> 128

Epoch 1/100
12228/12228 [==============================] - 8s 685us/step - loss: 7.4663e-04 - mae: 0.0187 - val_loss: 3.9406e-04 - val_mae: 0.0157
Epoch 2/100
12228/12228 [==============================] - 8s 635us/step - loss: 3.8103e-04 - mae: 0.0154 - val_loss: 3.6540e-04 - val_mae: 0.0151
Epoch 3/100
12228/12228 [==============================] - 8s 640us/step - loss: 3.7227e-04 - mae: 0.0152 - val_loss: 3.7141e-04 - val_mae: 0.0150
Epoch 4/100
12228/12228 [==============================] - 8s 640us/step - loss: 3.7029e-04 - mae: 0.0151 - val_loss: 3.6337e-04 - val_mae: 0.0151
Epoch 5/100
12228/12228 [==============================] - 8s 641us/step - loss: 3.6954e-04 - mae: 0.0151 - val_loss: 3.6951e-04 - val_mae: 0.0150
Epoch 6/100
12228/12228 [==============================] - 8s 641us/step - loss: 3.6847e-04 - mae: 0.0151 - val_loss: 3.7194e-04 - val_mae: 0.0154
Epoch 7/100
12228/12228 [==============================] - 8s 629us/step - loss: 3.6764e-04 - mae: 0.0151 - val_loss: 

In [17]:
# 데이터 불러오기
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values  # 훈련 데이터 입력 특성
y_train = data_train['FloodProbability'].values  # 훈련 데이터 타겟 변수
X_test = data_test.drop(['id'], axis=1).values  # 테스트 데이터 입력 특성

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping, model_checkpoint])

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

submission_df.head()

# CSV 파일로 저장
submission_df.to_csv('submission_ann.csv', index=False)

Epoch 1/100
17469/17469 [==============================] - 11s 641us/step - loss: 8.1638e-04 - mae: 0.0186 - val_loss: 3.7809e-04 - val_mae: 0.0154
Epoch 2/100
17469/17469 [==============================] - 10s 598us/step - loss: 3.7221e-04 - mae: 0.0152 - val_loss: 3.7409e-04 - val_mae: 0.0155
Epoch 3/100
17469/17469 [==============================] - 10s 600us/step - loss: 3.6645e-04 - mae: 0.0151 - val_loss: 3.6662e-04 - val_mae: 0.0148
Epoch 4/100
17469/17469 [==============================] - 11s 618us/step - loss: 3.6385e-04 - mae: 0.0150 - val_loss: 3.6553e-04 - val_mae: 0.0153
Epoch 5/100
17469/17469 [==============================] - 11s 621us/step - loss: 3.6270e-04 - mae: 0.0150 - val_loss: 3.5931e-04 - val_mae: 0.0149
Epoch 6/100
17469/17469 [==============================] - 11s 617us/step - loss: 3.6177e-04 - mae: 0.0149 - val_loss: 3.6579e-04 - val_mae: 0.0153
Epoch 7/100
17469/17469 [==============================] - 11s 612us/step - loss: 3.6129e-04 - mae: 0.0149 - val

,id,FloodProbability
0,1117957,0.576352
1,1117958,0.451702
2,1117959,0.454445
3,1117960,0.472365
4,1117961,0.472290


In [18]:

submission_df.to_csv('submission_ann.csv', index=False)

In [11]:
y_pred

array([[-0.0712075 ],
       [-0.07862947],
       [-0.16359659],
       ...,
       [ 0.00411901],
       [-0.00835801],
       [ 0.09335136]], dtype=float32)

In [3]:
data = pd.read_csv('train.csv') 

X = data.drop(['FloodProbability', 'id'], axis=1).values
y = data['FloodProbability'].values

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
# 은닉층 추가
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# 조기 종료 및 모델 체크포인트 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

# 모델 훈련
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping, model_checkpoint])

# 가장 좋은 모델 로드
model.load_weights('best_model.h5')

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/100
24456/24456 [==============================] - 22s 892us/step - loss: 8.7832e-04 - mae: 0.0195 - val_loss: 4.7691e-04 - val_mae: 0.0177
Epoch 2/100
24456/24456 [==============================] - 21s 870us/step - loss: 4.0456e-04 - mae: 0.0158 - val_loss: 4.4781e-04 - val_mae: 0.0169
Epoch 3/100
24456/24456 [==============================] - 21s 862us/step - loss: 3.9789e-04 - mae: 0.0157 - val_loss: 5.3456e-04 - val_mae: 0.0188
Epoch 4/100
24456/24456 [==============================] - 22s 886us/step - loss: 3.9461e-04 - mae: 0.0156 - val_loss: 4.5319e-04 - val_mae: 0.0171
Epoch 5/100
24456/24456 [==============================] - 22s 884us/step - loss: 3.9419e-04 - mae: 0.0156 - val_loss: 5.1101e-04 - val_mae: 0.0184
Epoch 6/100
24456/24456 [==============================] - 22s 893us/step - loss: 3.9369e-04 - mae: 0.0156 - val_loss: 4.9989e-04 - val_mae: 0.0182
Epoch 7/100
24456/24456 [==============================] - 21s 854us/step - loss: 3.9415e-04 - mae: 0.0156 - val

In [6]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

# 데이터 불러오기
data = pd.read_csv('train.csv') 

# 입력과 출력 데이터 나누기
X = data.drop(['FloodProbability','id'], axis=1).values
y = data['FloodProbability'].values

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam, RMSprop

# 모델 생성 함수
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# KerasRegressor 모델 생성
model = KerasRegressor(build_fn=create_model, verbose=0)

# 하이퍼파라미터 그리드
param_grid = {'batch_size': [16, 32, 64],
              'epochs': [50, 100, 200],
              'optimizer': [Adam(), RMSprop()]}

# 그리드 서치
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# 결과 출력
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\mm\AppData\Local\Temp\ipykernel_21460\1770470864.py:33: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model, verbose=0)


In [ ]:
# 데이터 불러오기
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values
y_train = data_train['FloodProbability'].values

X_test = data_test.drop(['id'], axis=1).values

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.2)

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

# CSV 파일로 저장
submission_df.to_csv('submission3.csv', index=False)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [20]:
from keras.models import Sequential
from keras.layers import Dense


# 입력과 출력 데이터 나누기
X = data.drop(['FloodProbability','id'], axis=1).values
y = data['FloodProbability'].values

# 데이터 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val))

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/30
24456/24456 [==============================] - 36s 1ms/step - loss: 6.5333e-04 - mae: 0.0175 - val_loss: 3.7532e-04 - val_mae: 0.0156
Epoch 2/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.7268e-04 - mae: 0.0152 - val_loss: 3.6519e-04 - val_mae: 0.0152
Epoch 3/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6843e-04 - mae: 0.0151 - val_loss: 3.7696e-04 - val_mae: 0.0156
Epoch 4/30
24456/24456 [==============================] - 36s 1ms/step - loss: 3.6711e-04 - mae: 0.0151 - val_loss: 3.5927e-04 - val_mae: 0.0148
Epoch 5/30
24456/24456 [==============================] - 36s 1ms/step - loss: 3.6605e-04 - mae: 0.0150 - val_loss: 3.6076e-04 - val_mae: 0.0149
Epoch 6/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6534e-04 - mae: 0.0150 - val_loss: 3.6253e-04 - val_mae: 0.0149
Epoch 7/30
24456/24456 [==============================] - 35s 1ms/step - loss: 3.6459e-04 - mae: 0.0150 - val_loss: 3.6094e-04 - v

In [3]:
# 입력과 출력 데이터 분할
X_train = data_train.drop(['FloodProbability', 'id'], axis=1).values
y_train = data_train['FloodProbability'].values

X_test = data_test.drop(['id'], axis=1).values

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 모델 훈련
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 테스트 데이터의 타겟 변수 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과를 데이터프레임으로 변환
submission_df = pd.DataFrame({'id': data_test['id'], 'FloodProbability': y_pred.flatten()})

# CSV 파일로 저장
submission_df.to_csv('submission2.csv', index=False)


Epoch 1/50
27949/27949 [==============================] - 41s 1ms/step - loss: 6.0796e-04 - mae: 0.0172 - val_loss: 3.9237e-04 - val_mae: 0.0160
Epoch 2/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.7322e-04 - mae: 0.0152 - val_loss: 3.8872e-04 - val_mae: 0.0150
Epoch 3/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6918e-04 - mae: 0.0151 - val_loss: 3.6249e-04 - val_mae: 0.0151
Epoch 4/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6679e-04 - mae: 0.0151 - val_loss: 3.6367e-04 - val_mae: 0.0152
Epoch 5/50
27949/27949 [==============================] - 38s 1ms/step - loss: 3.6489e-04 - mae: 0.0150 - val_loss: 3.6551e-04 - val_mae: 0.0153
Epoch 6/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6376e-04 - mae: 0.0150 - val_loss: 3.5923e-04 - val_mae: 0.0148
Epoch 7/50
27949/27949 [==============================] - 39s 1ms/step - loss: 3.6278e-04 - mae: 0.0150 - val_loss: 3.5992e-04 - v